In [1]:
import requests
import requests_cache
import pandas as pd
import itertools
from bs4 import BeautifulSoup
import pandas as pd
import math
import time
import shelve
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go

requests_cache.install_cache('craigslist_cache')
from data_function import *
df = get_all_search('san francisco bay area', 'owner','civic',2006,2011,0,100000)


In [2]:
df.head()

,post_title,mileage,price,condition,title_status,transmission,drive,location,body_text,link
0,2008 Honda Civic LX clean title LOW MILE,88700,6999,like new,clean,automatic,fwd,(hayward / castro valley),"2008 Honda Civic LX 2 Owner,clean title,passed...",https://sfbay.craigslist.org/eby/cto/d/2008-ho...
1,Honda Civic Hybrid 2006* Clean Title*V.Low miles,56650,5999,like new,clean,automatic,fwd,(hayward / castro valley),"2006 Honda Civic Hybrid LX 1 Owner,clean title...",https://sfbay.craigslist.org/eby/cto/d/honda-c...
2,2010 Honda Civic EX Coupe 2 Door,85250,8500,excellent,clean,automatic,fwd,(albany / el cerrito),I am the second owner and this car is in excel...,https://sfbay.craigslist.org/eby/cto/d/2010-ho...
3,Honda Civic si,,5000,,clean,manual,,(san leandro),"Hello there, I am selling a 2006 Honda Civic S...",https://sfbay.craigslist.org/eby/cto/d/honda-c...
4,"2007 HONDA CIVIC EX, 92K MILES, EXCELLENT COND...",92000,5250,excellent,salvage,automatic,,(san jose north),"2007 HONDA CIVIC EX, 92K MILS, AT, AM/FM SINGL...",https://sfbay.craigslist.org/sby/cto/d/2007-ho...


### Data Cleaning

In [3]:
df = df.replace(np.nan, 'Unknown', regex=True)
df.location = clean_location(df.location)
df.location = [x if len(x) != 0 else 'Unknown' for x in df.location] # Place Unknown if the location is empty
df.mileage = [str(x)+' miles' if x != '' else 'Unknown mileage' for x in df.mileage]
df.condition = ['condition:'+str(x) if x != '' else 'Unknown condition' for x in df.condition]


C:\Users\Jim\Desktop\Github\usedcaranalysis\data_function.py:215: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Jim\Desktop\Github\usedcaranalysis\data_function.py:224: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Jim\Desktop\Github\usedcaranalysis\data_function.py:220: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Jim\Desktop\Github\usedcaranalysis\data_function.py:221: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFr

In [5]:
with shelve.open('result/vars1') as db:
    db['Viz_df'] = df


## Data Visualization

**Please re-run this notebook and open the link of the below cell from your browser for the visualization**

In [ ]:
app = dash.Dash()

available_indicators = df['title_status'].unique()
list_price = list(np.linspace(0,100000,21))
#mileage column cannot be select and convert to str at the same time, must use concat 

text_scatter = pd.concat([df[['post_title', 'condition', 'link']],df['mileage'].astype(str)], axis=1).apply(lambda x: '<br>'.join(x), axis=1)

#color for the plot
palette = ["#053061", "#2166ac", "#4393c3", "#92c5de", "#d1e5f0",
           "#f7f7f7", "#fddbc7", "#f4a582", "#d6604d", "#b2182b", "#67001f"]
price = df["price"]
low = min(price)
high = max(price)
price_inds = [int(10*(x-low)/(high-low)) for x in price] 
df['price_colors'] = [palette[i] for i in price_inds]



app.layout = html.Div([
    html.Div([
        
    dcc.Markdown('# Craigslist Data Visualization'),
    dcc.Markdown('### Title Status Selection '),
        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='clean'),
        ],
        style={'width': '48%', 'display': 'inline-block'}),
    ]),

    dcc.Graph(id='indicator-graphic'),
    dcc.Markdown('### Price Range'),
    dcc.Slider(
        id='price--slider',
        min=0,
        max=200000,
        value=200000,
        step=None,
        marks={str(int(m)): str(int(m))  for m in list_price}
    )
#     html.Div(id='slider-output-container')
])

        
@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('xaxis-column', 'value'),
    dash.dependencies.Input('price--slider', 'value')])
#   dash.dependencies.Input('mileage--slider', 'value'
def update_graph(xaxis_column_name, price_value):
    dff = df[df['price'] <= int(price_value)]
    return {
        'data': [go.Scatter(
            x=dff[dff['title_status'] == xaxis_column_name]['location'],
            y=dff[dff['title_status'] == xaxis_column_name]['price'],
            mode='markers',
            text= text_scatter,
            hoverinfo= 'text',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'},
                'color' :  dff['price_colors']
            }
        )],
        'layout': go.Layout(
            title='Price vs Location',
            margin={'l': 40, 'b': 120, 't': 30, 'r': 50},
            height=400,
            hovermode='closest'
        )
    }

# @app.callback(
#     dash.dependencies.Output('slider-output-container', 'children'),
#     [dash.dependencies.Input('price--slider', 'value')])
# def update_output(value):
#     return 'The car price range is {} to {}'.format('0',value)

if __name__ == '__main__':
    app.run_server()


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
